In [3]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")

# Leemos una sola vez la base de datos por ejecución del programa
link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
data = pd.read_excel(link,'TODO CORREGIMIENTO')

def cleanBefore(data):  

    # eliminamos filas que no importan y asignamos la primera como cabecera
    data.drop([0], inplace=True)
    data.columns = data.iloc[0]
    data.drop([1], inplace=True)
    #eliminamos la primera fila que es un NA
    our_data_filter = data.iloc[1:]
    # extraemos un subconjunto que llegue solo hasta el dia de hoy:
    now = datetime.datetime.now()
    our_data_filter = our_data_filter[our_data_filter['Fecha'] < now]
    # filtramos desde el primer dia en el que haya recuperados:
    our_data_filter = our_data_filter[our_data_filter_un['NuevosRecuperadosacum'] > 0]
    # extraemos solo las columnas que nos interesan:
    our_data_filter = our_data_filter[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]
    return our_data_filter
    
def transAAC(data_transAAC):
    data_transAAC.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)
    data_transAAC['NuevosConfirmados'] = data_transAAC.to_numeric(data_transAAC['NuevosConfirmados'])
    data_transAAC['NuevosFallecidos'] = data_transAAC.to_numeric(data_transAAC['NuevosFallecidos'])
    data_transAAC['NuevosRecuperados'] = data_transAAC.to_numeric(data_transAAC['NuevosRecuperados'])
    data_transAAC['Poblacion'] = data_transAAC.to_numeric(data_transAAC['Poblacion'])
    data_transAAC['NuevosConfirmados'] = data_transAAC['NuevosConfirmados']+data_transAAC['NuevosFallecidos']
    data_transAAC['NuevosConfirmadosacum'] = data_transAAC['NuevosConfirmados'].cumsum()
    data_transAAC['NuevosRecuperadosacum'] = data_transAAC['NuevosRecuperados'].cumsum()
    data_transAAC["Susceptibles"] = data_transAAC['Poblacion']-data_transAAC['NuevosConfirmadosacum']-data_transAAC['NuevosRecuperadosacum']
    data_transAAC["pob_log_Susceptibles"] = data_transAAC['Poblacion']*np.log(data_transAAC["Susceptibles"])
    return data_transAAC

def calculate(data_cal):
    lista_R0 = []
    y = our_data_filter_un_final["pob_log_Susceptibles"]
    for i in range(len(y)):
        y = our_data_filter_un_final["pob_log_Susceptibles"]
        x = our_data_filter_un_final["NuevosRecuperadosacum"]
        yy = y.head(i+1)
        xx = x.head(i+1)
        # Creamos el objeto de Regresión Lineal
        regresion_lineal = linear_model.LinearRegression()
        regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
        r0 = -regresion_lineal.coef_
        lista_R0.append(r0)
    our_data_filter_un_final["r0"] = lista_R0
    return our_data_filter_un_final

def cleanAfter(data_second_r0):
    x = data_second_r0['r0'].str.replace("]]", "")
    y = x.str.replace("[", "")
    z_dataframe = pd.DataFrame(z)
    del data_second_r0['r0']
    data_second_r0['r0'] = z_dataframe
    return data_second_r0

def saveParcial(dt):
    #Dejar los generados archivos en la misma carpeta
    dt.to_csv('test.csv', mode='a', index = False, header=False)
    return 

def R0PanamaPorPartes():
    # ?
    # data = recuperarData()
    for corregimiento in data["Corregimiento"].unique():
        saveParcial(cleanAfter(calculate(transAAC(cleanBefore(data[data["Corregimiento"] == corregimiento])))))
    return

R0PanamaPorPartes()

KeyboardInterrupt: 